In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.ensemble import IsolationForest
from sklearn.manifold import TSNE
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import shap
import json
from prettytable import PrettyTable
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

In [5]:
# -------------------------
# 1. DATA LOADING & CLEANING
# -------------------------
def load_and_clean(file_path):
    df = pd.read_excel(file_path)
    df = df.drop(columns=['FID', 'Shape *', 'gid', 'objectid', 'toposheet'], errors='ignore')
    essential_elements = ['X', 'Y', 'Si02_%', 'Al2O3_%', 'Fe2O3_%', 'TiO2_%',
                        'MgO_%', 'MnO_%', 'Na2O_%', 'K2O_%', 'Ni_ppm', 'Co_ppm',
                        'Cu_ppm', 'Zn_ppm', 'Au_ppb', 'La_ppm', 'Ce_ppm', 'Zr_ppm',
                        'Hf_ppm', 'Pt_ppb', 'Pd_ppb']
    df = df[essential_elements].dropna()
    iso = IsolationForest(contamination=0.05, random_state=42)
    outliers = iso.fit_predict(df.select_dtypes(include=np.number))
    df = df[outliers == 1]
    pt = PowerTransformer()
    num_cols = df.select_dtypes(include=np.number).columns
    df[num_cols] = pt.fit_transform(df[num_cols])
    print(f"✅ Cleaned Data: {df.shape[0]} samples, {df.shape[1]} features")
    return df, pt


In [6]:
# -------------------------
# 2. MINERAL TARGETING FEATURES
# -------------------------
def create_mineral_features(df):
    df['Ni/Cu_Ratio'] = df['Ni_ppm'] / (df['Cu_ppm'] + 1e-6)
    df['PGE_Index'] = (df['Pt_ppb'] + df['Pd_ppb']) / 1000
    df['LREE/HREE'] = (df['La_ppm'] + df['Ce_ppm']) / (df.get('Yb_ppm', 1) + 1e-6)
    df['Argillic_Index'] = (df['Al2O3_%'] + df['K2O_%']) / (df['Si02_%'] + 1e-6)
    df['Chlorite_Index'] = df['MgO_%'] / (df['Fe2O3_%'] + 1e-6)
    df['Zr/Hf_Depth'] = df['Zr_ppm'] / (df['Hf_ppm'] + 1e-6)
    print("✅ Added 5 genetic model features")
    return df


In [7]:
# -------------------------
# 3. DATA SPLITTING (Multi-label)
# -------------------------
def split_data(df, test_size=0.2, val_size=0.2):
    target_elements = ['Cu_ppm', 'Ni_ppm', 'Co_ppm', 'Zn_ppm', 'La_ppm', 'Ce_ppm', 'Au_ppb', 'Pt_ppb']
    for el in target_elements:
        threshold = df[el].quantile(0.85)
        df[f"{el}_label"] = (df[el] > threshold).astype(int)
    target_cols = [f"{el}_label" for el in target_elements]
    X = df.drop(columns=target_cols)
    y = df[target_cols]
    X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=val_size, random_state=42)
    return X_train, X_val, X_test, y_train, y_val, y_test, target_cols

In [8]:
# -------------------------
# 4. MINERAL PREDICTION MODEL (Updated with LayerNorm)
# -------------------------
class MineralNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.attention = nn.Sequential(
            nn.Linear(input_dim, input_dim),
            nn.Softmax(dim=1)
        )
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.LayerNorm(256),  # Changed from BatchNorm to LayerNorm
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 8),
            nn.Sigmoid()
        )

    def forward(self, x):
        attn_weights = self.attention(x)
        x = x * attn_weights
        return self.classifier(x), attn_weights

In [9]:

# -------------------------
# 5. TRAINING FUNCTIONS (Updated with drop_last=True)
# -------------------------
def train_model(X_train, y_train, X_val, y_val):
    train_data = TensorDataset(torch.tensor(X_train.values, dtype=torch.float32), 
                             torch.tensor(y_train.values, dtype=torch.float32))
    val_data = TensorDataset(torch.tensor(X_val.values, dtype=torch.float32), 
                           torch.tensor(y_val.values, dtype=torch.float32))
    
    # Added drop_last=True to prevent single-sample batches
    train_loader = DataLoader(train_data, batch_size=32, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_data, batch_size=32, drop_last=True)
    
    model = MineralNN(X_train.shape[1])
    criterion = nn.BCELoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
    
    best_val_loss = float('inf')
    train_losses, val_losses = [], []
    
    for epoch in range(50):
        model.train()
        train_loss = 0
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            pred, _ = model(batch_X)
            loss = criterion(pred, batch_y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                pred, _ = model(batch_X)
                val_loss += criterion(pred, batch_y).item()
                
        train_losses.append(train_loss/len(train_loader))
        val_losses.append(val_loss/len(val_loader))
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            
        print(f"Epoch {epoch+1:02d}: Train Loss = {train_losses[-1]:.4f}, Val Loss = {val_losses[-1]:.4f}")
    
    return model, train_losses, val_losses

def plot_training_history(train_losses, val_losses):
    plt.figure(figsize=(8, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training History')
    plt.legend()
    plt.grid(True)
    plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
    plt.close()

def generate_3d_maps(df, model, scaler, feature_names, target_cols):
    X_raw = df[feature_names].copy()
    X_scaled = scaler.transform(X_raw)
    
    with torch.no_grad():
        probs, _ = model(torch.tensor(X_scaled, dtype=torch.float32))
    
    minerals = ['Cu_ppm', 'Ni_ppm', 'Co_ppm', 'Zn_ppm', 'La_ppm', 'Ce_ppm', 'Au_ppb', 'Pt_ppb']
    for i, mineral in enumerate(minerals):
        fig = go.Figure(data=[
            go.Scatter3d(
                x=df['X'], y=df['Y'], z=-df['Zr/Hf_Depth']*100,
                mode='markers',
                marker=dict(
                    size=5,
                    color=probs[:, i].numpy(),
                    colorscale='Hot',
                    cmin=0,
                    cmax=1,
                    opacity=0.8,
                    colorbar=dict(title='Probability')
                ),
                text=df[mineral]
            )
        ])
        fig.update_layout(
            scene=dict(
                xaxis_title='Longitude',
                yaxis_title='Latitude',
                zaxis_title='Depth Proxy (Zr/Hf)'
            ),
            title=f"3D Prospectivity Map — {mineral}"
        )
        fig.write_html(f"3d_prospectivity_{mineral}.html")

def plot_mineral_clusters(X, y, feature_names):
    tsne = TSNE(n_components=2, random_state=42)
    tsne_results = tsne.fit_transform(X)
    
    dominant_label = y.idxmax(axis=1)
    fig = go.Figure()
    
    for label in dominant_label.unique():
        mask = dominant_label == label
        fig.add_trace(go.Scatter(
            x=tsne_results[mask, 0],
            y=tsne_results[mask, 1],
            mode='markers',
            name=label.replace('_label', ''),
            marker=dict(size=5),
            text=dominant_label[mask]
        ))
    
    fig.update_layout(
        title="Mineral Composition Clusters (t-SNE)",
        xaxis_title="t-SNE 1",
        yaxis_title="t-SNE 2"
    )
    fig.write_html("mineral_clusters.html")

def plot_confusion_matrices(y_true, y_pred, target_cols):
    for i, col in enumerate(target_cols):
        cm = confusion_matrix(y_true[:, i], y_pred[:, i])
        plt.figure(figsize=(4, 3))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
        plt.title(f"Confusion Matrix: {col}")
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.tight_layout()
        plt.savefig(f"confusion_matrix_{col}.png", dpi=300, bbox_inches='tight')
        plt.close()

def generate_shap_analysis(model, X_train_scaled, feature_names, target_cols):
    # Use a subset for faster computation
    background = shap.utils.sample(X_train_scaled.values, 100)
    explain_data = shap.utils.sample(X_train_scaled.values, 200)
    
    def model_wrapper(x):
        x_tensor = torch.tensor(x, dtype=torch.float32)
        with torch.no_grad():
            preds, _ = model(x_tensor)
        return preds.numpy()
    
    explainer = shap.Explainer(model_wrapper, background)
    shap_values = explainer(explain_data)
    
    # Global SHAP summary for each target
    for i, target in enumerate(target_cols):
        plt.figure()
        shap.summary_plot(shap_values[:, :, i].values, 
                         features=explain_data, 
                         feature_names=feature_names,
                         show=False)
        plt.title(f"SHAP Summary - {target}")
        plt.tight_layout()
        plt.savefig(f"shap_summary_{target}.png", dpi=300, bbox_inches='tight')
        plt.close()
    
    # Force plots for first 3 samples
    for i, target in enumerate(target_cols):
        for j in range(3):
            plt.figure()
            shap.plots.force(shap_values[j, :, i], 
                           features=explain_data[j], 
                           feature_names=feature_names,
                           matplotlib=True, show=False)
            plt.title(f"SHAP Force Plot - {target} (Sample {j+1})")
            plt.tight_layout()
            plt.savefig(f"shap_force_{target}_sample{j+1}.png", dpi=300, bbox_inches='tight')
            plt.close()
    
    # Comprehensive SHAP summary
    plt.figure(figsize=(12, 8))
    shap_values_all = np.stack([shap_values[:, :, i].values for i in range(len(target_cols))])
    feature_importance = np.abs(shap_values_all).mean(axis=1)
    norm_importance = feature_importance / feature_importance.sum(axis=1, keepdims=True)
    
    plot_df = pd.DataFrame(
        norm_importance.T,
        index=feature_names,
        columns=target_cols
    )
    plot_df['total'] = plot_df.sum(axis=1)
    plot_df = plot_df.sort_values('total', ascending=False).drop('total', axis=1)
    
    sns.heatmap(
        plot_df,
        cmap='viridis',
        annot=True,
        fmt='.2f',
        cbar_kws={'label': 'Normalized SHAP Importance'},
        linewidths=0.5
    )
    plt.title('Comprehensive SHAP Feature Importance')
    plt.xlabel('Mineral Targets')
    plt.ylabel('Geochemical Features')
    plt.tight_layout()
    plt.savefig('comprehensive_shap_summary.png', dpi=300, bbox_inches='tight')
    plt.close()


In [10]:
# -------------------------
# MAIN WORKFLOW
# -------------------------
if __name__ == "__main__":
    # 1. Load and clean data
    df, pt = load_and_clean("/kaggle/input/my-dataset/NGCM-Stream-Sediment-Analysis-Updated.xlsx")
    df = create_mineral_features(df)
    
    # 2. Split data
    X_train, X_val, X_test, y_train, y_val, y_test, target_cols = split_data(df)
    
    # 3. Scale data
    scaler = StandardScaler().fit(X_train.select_dtypes(include=np.number))
    X_train_scaled = X_train.copy()
    X_val_scaled = X_val.copy()
    X_test_scaled = X_test.copy()
    X_train_scaled[X_train.select_dtypes(include=np.number).columns] = scaler.transform(X_train.select_dtypes(include=np.number))
    X_val_scaled[X_val.select_dtypes(include=np.number).columns] = scaler.transform(X_val.select_dtypes(include=np.number))
    X_test_scaled[X_test.select_dtypes(include=np.number).columns] = scaler.transform(X_test.select_dtypes(include=np.number))
    feature_names = X_train.select_dtypes(include=np.number).columns.tolist()
    
    # 4. Train model
    model, train_losses, val_losses = train_model(X_train_scaled, y_train, X_val_scaled, y_val)
    plot_training_history(train_losses, val_losses)
    
    # 5. Evaluate model
    with torch.no_grad():
        # Validation metrics
        val_probs, _ = model(torch.tensor(X_val_scaled.values, dtype=torch.float32))
        val_pred = (val_probs > 0.5).float().numpy()
        val_true = y_val.values
        val_acc = (val_pred == val_true).mean()
        print(f"\n📈 Validation Set Accuracy (per label mean): {val_acc:.4f}")
        
        # Test metrics
        test_probs, _ = model(torch.tensor(X_test_scaled.values, dtype=torch.float32))
        test_pred = (test_probs > 0.5).float().numpy()
        test_true = y_test.values
        test_acc = (test_pred == test_true).mean()
        print(f"📈 Test Set Accuracy (per label mean): {test_acc:.4f}")
    
    # 6. Print classification report
    print("\n📊 Multi-label Test Set Performance:")
    print(classification_report(y_test, test_pred, target_names=target_cols))
    
    # 7. Print AUC-ROC scores
    print("AUC-ROC Scores:")
    for i, col in enumerate(target_cols):
        try:
            auc_score = roc_auc_score(y_test[col], test_probs.numpy()[:, i])
            print(f"{col}: {auc_score:.3f}")
        except ValueError:
            print(f"{col}: Not computable (only one class present)")
    
    # 8. Generate visualizations
    generate_3d_maps(df, model, scaler, feature_names, target_cols)
    plot_mineral_clusters(X_test_scaled.values, y_test, feature_names)
    plot_confusion_matrices(test_true, test_pred, target_cols)
    generate_shap_analysis(model, X_train_scaled, feature_names, target_cols)
    
    # 9. Save model parameters
    params = {
        "data_preprocessing": {
            "initial_samples": df.shape[0],
            "final_features": len(feature_names),
            "outlier_method": "IsolationForest (5%)",
            "transformations": ["PowerTransformer", "StandardScaler"]
        },
        "model": {
            "architecture": "AttentionNN(256-128-8)",
            "optimizer": "AdamW (lr=0.001, wd=0.01)",
            "training_samples": len(X_train),
            "validation_accuracy": float(val_acc),
            "test_accuracy": float(test_acc),
            "output_targets": target_cols
        }
    }
    with open("model_params.json", "w") as f:
        json.dump(params, f, indent=4)
    
    print("\n✅ All requested outputs generated:")
    print("- 3d_prospectivity_<mineral>.html (8 files)")
    print("- mineral_clusters.html")
    print("- training_history.png")
    print("- confusion_matrix_<label>.png (8 files)")
    print("- comprehensive_shap_summary.png")
    print("- model_params.json (now includes accuracy metrics)")
    print("- best_model.pth")
    print("- shap_summary_<target>.png (8 files)")
    print("- shap_force_<target>_sampleX.png (24 files)")

✅ Cleaned Data: 9503 samples, 21 features
✅ Added 5 genetic model features
Epoch 01: Train Loss = 0.2196, Val Loss = 0.1204
Epoch 02: Train Loss = 0.0928, Val Loss = 0.0707
Epoch 03: Train Loss = 0.0620, Val Loss = 0.0513
Epoch 04: Train Loss = 0.0475, Val Loss = 0.0452
Epoch 05: Train Loss = 0.0397, Val Loss = 0.0392
Epoch 06: Train Loss = 0.0369, Val Loss = 0.0386
Epoch 07: Train Loss = 0.0312, Val Loss = 0.0297
Epoch 08: Train Loss = 0.0300, Val Loss = 0.0305
Epoch 09: Train Loss = 0.0269, Val Loss = 0.0279
Epoch 10: Train Loss = 0.0234, Val Loss = 0.0278
Epoch 11: Train Loss = 0.0253, Val Loss = 0.0245
Epoch 12: Train Loss = 0.0206, Val Loss = 0.0230
Epoch 13: Train Loss = 0.0203, Val Loss = 0.0249
Epoch 14: Train Loss = 0.0198, Val Loss = 0.0352
Epoch 15: Train Loss = 0.0190, Val Loss = 0.0207
Epoch 16: Train Loss = 0.0178, Val Loss = 0.0270
Epoch 17: Train Loss = 0.0168, Val Loss = 0.0208
Epoch 18: Train Loss = 0.0181, Val Loss = 0.0245
Epoch 19: Train Loss = 0.0163, Val Loss = 0

PermutationExplainer explainer: 201it [00:22,  6.33it/s]                         



✅ All requested outputs generated:
- 3d_prospectivity_<mineral>.html (8 files)
- mineral_clusters.html
- training_history.png
- confusion_matrix_<label>.png (8 files)
- comprehensive_shap_summary.png
- model_params.json (now includes accuracy metrics)
- best_model.pth
- shap_summary_<target>.png (8 files)
- shap_force_<target>_sampleX.png (24 files)


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>